In [15]:
import pandas
import math
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm
from scipy.stats import norm
from statsmodels.sandbox.regression.gmm import GMM
from statsmodels.base.model import GenericLikelihoodModel
from scipy.optimize import minimize

In [9]:
#load data into memory
data3 = np.genfromtxt('data3.dat', delimiter='  ')
print(data3.shape)
#test to see if other data works
drawsml = np.genfromtxt('drawsml.dat', delimiter='  ')

(50, 3)


In [10]:
# 50 markets
p = data3[:,0]
x1 = data3[:,1]
x2 = data3[:,2]

In [11]:
print(p.shape)

(50,)


In [12]:
print(drawsml[:, 1].shape)

(50,)


In [39]:
def loglike(params, *args):
    loglike = 0
    p, x1, x2, drawsml = args
    s = drawsml.shape[1]
    theta1, theta2, sigma = params
    for i in range(s):
        mc1 = np.exp(theta1 + theta2*x1 + sigma*drawsml[:, i])
        n = len(p)
        z = np.maximum(3*p - 100 - mc1, [1e-5]*n)
        e = np.log(z) - theta1 - theta2*x2
        loglike += -n/2*np.log(2*np.pi*sigma**2) - 1/(2*sigma**2)*sum(e**2)
    return -loglike/s

In [43]:
res = minimize(loglike, [1,1,1], args=(p, x1, x2, drawsml), bounds=((None, None), (None, None), (1e-10, None)))

In [44]:
res

  message: CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH
  success: True
   status: 0
      fun: 158.96945937415813
        x: [-1.734e+00  5.276e-01  5.861e+00]
      nit: 28
      jac: [ 1.137e-05 -2.842e-06 -2.842e-06]
     nfev: 124
     njev: 31
 hess_inv: <3x3 LbfgsInvHessProduct with dtype=float64>

In [28]:
np.maximum([1,2,3], [2,2,2])

array([2, 2, 3])

## SMM

In [13]:
# simulated GMM
# fix a numpy random seed
np.random.seed(1234)
sim = 20
e1 = np.random.normal(0, 1, (50,sim))
e2 = np.random.normal(0, 1, (50,sim))

In [14]:
print(e1[:, 1].shape)

(50,)


In [21]:
# model moments is the average of the 20 simulated moments
# return shape: 4x1
def model_moment(p, x1, x2, e1, e2, t1, t2, sigma, sim):
    mom1 = []
    mom2 = []
    mom3 = []
    mom4 = []
    for i in range(sim):
        mc1 = np.exp(t1 + t2*x1 + sigma*e1[:, i])
        mc2 = np.exp(t1 + t2*x2 + sigma*e2[:, i])
        phat = 100 - (200 - mc1 - mc2)/3
        moment1 = p - phat
        mom1.append(moment1)
        moment2 = moment1 * x1
        mom2.append(moment2)
        moment3 = moment1 * x2
        mom3.append(moment3)
        moment4 = p**2 - phat**2
        mom4.append(moment4)
    m1 = np.array(mom1).mean()
    m2 = np.array(mom2).mean()
    m3 = np.array(mom3).mean()
    m4 = np.array(mom4).mean()
    moments = np.array([[m1], [m2], [m3], [m4]])
    return moments

In [22]:
def gmm(params, *args):
    p, x1, x2, e1, e2, sim, W = args
    t1, t2, sigma = params
    moments = model_moment(p, x1, x2, e1, e2, t1, t2, sigma, sim)
    return moments.T @ W @ moments

In [41]:
W = np.eye(4)
res = minimize(gmm, [1,1,0.07], args=(p, x1, x2, e1, e2, sim, W), method='L-BFGS-B', bounds=((None, None), (None, None), (None, None)))

In [42]:
print(res.x)

[1.00443762 1.00004825 0.07043053]
